# Program Assignment #2 - 
# Decision Tree & Random Forest & KNN & PCA
---

## Name: 李勝維
## Student ID: 0711239

---

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(0)

# Load data

In [88]:
from sklearn.preprocessing import OneHotEncoder
# load data
raw_data = pd.read_csv("data/student-mat.csv", sep=";").values

# shuffle data
np.random.shuffle(raw_data)
x, y = raw_data[..., :-1], raw_data[..., -1, None]
Categorical_Features = [0, 1, 3, 4, 5, 8, 9, 10, 11, 15, 16, 17, 18, 19, 20, 21, 22]
Numerical_Features = [2, 6, 7, 12, 13, 14, 23, 24, 25, 26, 27, 28, 29, 30, 31]

# transform x to one-hot
x_encoder = OneHotEncoder(sparse=False)
x_cat = x_encoder.fit_transform(x[..., Categorical_Features])
x_num = x[..., Numerical_Features]
x = np.hstack((x_cat, x_num))

# Principal components analysis
transforms training set and test set based on training set

In [ ]:
from sklearn.decomposition import PCA
def pca_transform(train, test, k=10):
    pca = PCA(n_components=k)
    train = pca.fit_transform(train)
    test = pca.transform(test)
    return train, test

# Model Construction
Algorithm behind:  
DecisionTreeClassifier from sklearn.tree uses CART (Classification and Regression Trees) algorithm, which is similar to C4.5 algorithm, but it supports numerical value (regression) and constructs binary trees using feature and its threshold which yields the largest information gain at each node.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
DTclf = DecisionTreeClassifier(random_state=0)
RFclfs = [ # with different size of trees
    RandomForestClassifier(random_state=0, n_estimators=4),
    RandomForestClassifier(random_state=0, n_estimators=8),
    RandomForestClassifier(random_state=0, n_estimators=16)
]
KNNclfs =[ # with different k's
    KNeighborsClassifier(n_neighbors=3),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=7),
]

# Results

In [ ]:
# part 1 compare with / without PCA on three kinds of model